# 📦 Project: The Shipping Crisis (Saudi Retail Case Study)
## 📋 The Scenario
You have been hired by a major Saudi-based e-commerce platform. Management has noticed a significant decline in customer satisfaction scores. Initial reports suggest that a massive number of packages are not reaching customers on time.

**The Management's Question:**
*"We are losing money and customers. We need to know: Is this a logistics problem (Mode of Shipment), a product problem (Weight/Cost), or a strategy problem (Discounts/Priority)? Find the root cause and build a system to save our customers."*

## Phase 1: Data Understanding
In this phase, explore the `Saudi_Retail_Logistics_Data.csv` to find out **why** delays are happening.

* **Step 1.1:** Load the dataset.
* **Step 1.2:** Perform statistical profiling (`.describe()`, `.info()`).
* **Step 1.3:** Create visualizations (Histograms/Scatter) to find the problem.

| Feature Name         | Type       | Description                                                                 | Values / Examples                               |
| :------------------- | :--------- | :-------------------------------------------------------------------------- | :--------------------------------------------- |
| Tracking_ID          | String     | A unique, encrypted alphanumeric identifier for each shipment.              | E3B0C44298FC, 1C2D3E4F5G6H                     |
| Warehouse_Zone       | Categorical| The regional distribution hub handling the order.                           | North_Hub, South_Hub, East_Hub, West_Hub, Central_Port |
| Mode_of_Shipment     | Categorical| The logistics method used to transport the product.                         | Premium_Air, Standard_Sea, Ground_Freight      |
| Customer_Inquiries   | Integer    | Number of times the customer contacted support regarding this order.        | 2, 3, 4, etc.                                  |
| Customer_Score       | Integer    | The rating provided by the customer to the store before shipment (1 is lowest, 5 is highest).            | 1, 2, 3, 4, 5                                  |
| Product_Cost_SAR     | Float      | The value of the item in Saudi Riyals (SAR).                                | 450.00, 1125.50, etc.                          |
| Previous_Orders      | Integer    | Number of successful orders this customer has placed previously.            | 0, 1, 2, 3, etc.                               |
| Product_Priority     | Categorical| The priority level assigned to the product for dispatch.                    | low, medium, high                              |
| Gender               | Categorical| The gender of the customer.                                                 | F, M                                           |
| Discount_offered     | Integer    | The discount amount applied to this specific order.                         | 1, 10, 65, etc.                                |
| Product_Weight_g     | Integer    | The weight of the package in grams.                                         | 1200, 4500, etc.                               |
| is_delayed           | Binary     | Target Variable: Indicates if the shipment arrived past the deadline.       | 1 (Delayed), 0 (On Time)                       |

### **Step 1.1:** Load the dataset.


In [ ]:
from pathlib import Path
import pandas as pd
local_file = Path('Saudi_Retail_Logistics_Data.csv')

if not local_file.exists():
    print("Downloading data...")
    df = pd.read_csv('https://drive.google.com/uc?export=download&id=1vayY_XKPwe5lOaq0VNia22O4DkDVXmzv')
    df.to_csv(local_file, index=False)
else:
    print("Loading data from cache...")
    df = pd.read_csv(local_file)
df.head()

Loading data from cache...


,Tracking_ID,Warehouse_Zone,Mode_of_Shipment,Customer_Inquiries,Customer_Score,Product_Cost_SAR,Previous_Orders,Product_Priority,Gender,Discount_offered,Product_Weight_g,is_delayed
0,D66B138C54AB,West_Hub,Ground_Freight,4,1,978.75,2,medium,F,9,4158,0
1,8FF9538E65E6,Central_Port,Standard_Sea,3,3,607.50,2,low,F,26,1659,1
2,D098284FA376,South_Hub,Standard_Sea,3,2,828.75,2,medium,M,9,4466,0
3,9202E9FDFF40,West_Hub,Standard_Sea,3,5,588.75,4,medium,F,2,4640,0
4,16B30490A644,North_Hub,Premium_Air,2,5,1020.00,2,low,M,24,3638,1


### **Step 1.2:** Perform statistical profiling (`.describe()`, `.info()`).


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8799 entries, 0 to 8798
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Tracking_ID         8799 non-null   object 
 1   Warehouse_Zone      8799 non-null   object 
 2   Mode_of_Shipment    8799 non-null   object 
 3   Customer_Inquiries  8799 non-null   int64  
 4   Customer_Score      8799 non-null   int64  
 5   Product_Cost_SAR    8799 non-null   float64
 6   Previous_Orders     8799 non-null   int64  
 7   Product_Priority    8799 non-null   object 
 8   Gender              8799 non-null   object 
 9   Discount_offered    8799 non-null   int64  
 10  Product_Weight_g    8799 non-null   int64  
 11  is_delayed          8799 non-null   int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 825.0+ KB


This output shows that the dataset contains 8,799 rows and 12 columns with no missing values, indicating good data quality. It also confirms the correct data types, with a mix of categorical and numerical variables, and a binary target variable.

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Customer_Inquiries,8799.0,4.049096,1.138649,2.0,3.0,4.0,5.00,7.0
Customer_Score,8799.0,2.997386,1.411194,1.0,2.0,3.0,4.00,5.0
Product_Cost_SAR,8799.0,788.363877,180.192439,360.0,637.5,802.5,941.25,1162.5
Previous_Orders,8799.0,3.581998,1.532722,2.0,3.0,3.0,4.00,10.0
Discount_offered,8799.0,13.366974,16.147015,1.0,4.0,7.0,10.00,65.0
Product_Weight_g,8799.0,3631.227867,1634.180615,1001.0,1838.0,4140.0,5045.00,7846.0
is_delayed,8799.0,0.596659,0.490596,0.0,0.0,1.0,1.00,1.0


This summary provides the statistical distribution of numerical features, including mean, spread, and range. It reveals that about 60% of shipments are delayed, product weights and costs vary widely, discounts are highly skewed, and customer scores are centered around an average rating of 3.

### **Step 1.3:** Create visualizations (Histograms/Scatter) to find the problem.

### Delay Distribution

Evaluates whether shipment delays are isolated incidents or a systemic problem. High delay rates indicate a crisis.

In [7]:
df["is_delayed"].value_counts(normalize=True)

is_delayed
1    0.596659
0    0.403341
Name: proportion, dtype: float64

### Is this a logistics problem?

In [ ]:
import plotly.express as px

fig = px.histogram(
    df,
    x="Mode_of_Shipment",
    color="is_delayed",
    barnorm="fraction",   # or "fraction"
    barmode="stack",
    title="On-time vs Delayed Proportions by Mode of Shipment",
    labels={"is_delayed": "Delivery Status"}
)

fig.update_yaxes(title="Percent of Shipments")
fig.show()


## Phase 2: Feature Engineering & Baseline
Prepare the data for Machine Learning.

## Phase 3: Training, Evaluation & Strategy
Build your solution and define your business interventions.

* **Step 3.1:** Train a Classifier.
* **Step 3.2:** Generate a Classification Report and Confusion Matrix.
* **Step 3.3:** **CRITICAL:** List 3 business solution will save our customers based on your model (you can use a 5% Coupon).

In [ ]:
# Model training and evaluation code here


### 📝 Business Strategy Recommendations
1. 
2. 
3. 

## Phase 4: The Business Value Calculator
Calculate the SAR impact of your model using the following values:
* **+ SAR 75.00**: True Positive (Saved Customer)
* **- SAR 18.75**: False Positive (5% Wasted Coupon)
* **- SAR 187.50**: False Negative (Lost Lifetime Value)

In [ ]:
# Calculate and print the Total Business Value in SAR
